# Forward and backward passess
using gradients in backward propagationn to update weights on a tensor

## Setup and getting the data

In [6]:
# Downloading the MNIST dataset

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [8]:
# Boilerplate code to load all the relevant libraries and put the data into tensors

import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## The basic architecture

Using SGD, not cross-entropy and then evaluating the results using MSE. It is a hacky way as proof of concept

In [12]:
n,m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [13]:
# number of hidden layers (arbitrarily chosen)

nh = 50

In [14]:
# Initialise all the weights and biases on our new basic neural network

w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [17]:
def lin(x, w, b): return x@w + b

In [18]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [21]:
# Relu function clamping, so changing all values less than zero to zero

def relu(x): return x.clamp_min(0.)

In [22]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [23]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)


In [25]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [26]:
res

tensor([[  25.75],
        [ -13.06],
        [-114.79],
        ...,
        [ -67.44],
        [ -74.48],
        [ -60.19]])

# MSE, defining a loss function

In [27]:
y_valid.shape, res.shape

(torch.Size([10000]), torch.Size([10000, 1]))

In [28]:
# To subtract these from each other we need to add the index collumn, or use the squeeze method to remove the trailing unit vector
res.squeeze().shape

torch.Size([10000])

In [34]:
# Turn targets of training and validation sets into floats as we are using MSE
y_train,y_valid = y_train.float(),y_valid.float()

# Make our first set of predictions using our model
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [35]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [37]:
mse(preds, y_train)

tensor(4308.76)

## Gradients and backward pass

To do here:
- figure out what all the individual lines of code do and why
- Experiment with the python debugger 
import pdb; pdb.set_trace() is the code that places a breakpoint. With this the code stops running and you can use various commands to assess the states of the various variables at that point

In [ ]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
#     Note that the t() method transposes the tensor
    w.g = >

In [ ]:
def forward_and_backward(inp, target):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0] - target
    loss = diff.pow(2).mean() 